In [1]:
import pandas as pd
import numpy as np

# 1. Carregar a base original (Morumbi)

In [2]:
# Carrega o arquivo de entrada (arquivo organizado em `filling_Ceps/`)
df = pd.read_csv('../filling_Ceps/Elvira Brandão Morumbi - Euvira Brandão Dados ADS_coords_corrigidas_com_enderecos.csv')

# 2. Tratar a renda para número

In [3]:
import re

def parse_renda_seguro(x):
    if pd.isna(x):
        return np.nan
    x = str(x)

    # remove qualquer símbolo que não seja número, vírgula ou ponto
    x = re.sub(r"[^0-9,\.]", "", x)

    # caso venha no formato errado tipo "19.448.421.299.999.900"
    # mantemos somente o último grupo decimal
    if x.count(".") > 1:
        # remove TODOS os pontos; eles NÃO representam milhar
        x = x.replace(".", "")

    # agora troca vírgula por ponto
    x = x.replace(",", ".")

    try:
        return float(x)
    except:
        return np.nan


df["renda_media_num"] = df["renda_media"].apply(parse_renda_seguro)

# 3. Criar Total_0_9 (0–4 + 5–9 anos)

In [4]:
df["Total_0_9"] = (
    df["v01031_0_4anos"].fillna(0) +
    df["v01032_5_9anos"].fillna(0)
)

# 4. Corrigir renda para 2025 (inflação)

In [5]:
inflation_factor = 1.155
df["renda_atualizada_2025"] = df["renda_media_num"] * inflation_factor

# 5. Score linha a linha (informativo, não consolidado)

In [6]:
df["score_trafego_2025"] = df["renda_atualizada_2025"] * df["Total_0_9"]

# 6. Consolidar por CEP (SEM SOMAR — usar média!)

In [7]:
df_cep = (
    df.groupby("CEP", as_index=False)
      .agg({
          "Bairro": lambda x: x.mode().iat[0] if not x.mode().empty else x.iloc[0],
          "renda_atualizada_2025": "median",
          "Total_0_9": "median",
          "populacao_total": "median"
      })
)

In [8]:
# Renomear colunas para refletir que são médias, não totais
df_cep = df_cep.rename(columns={
    "renda_atualizada_2025": "renda_mediana_2025",
    "Total_0_9": "mediana_criancas_0_9",
    "populacao_total": "populacao_mediana"
})

# 7. Score final no nível do CEP (correto)

In [9]:
df_cep["score_trafego_2025"] = (
    df_cep["renda_mediana_2025"] * df_cep["mediana_criancas_0_9"]
)

# 8. Ranking final

In [10]:
top_ceps = df_cep.sort_values("score_trafego_2025", ascending=False)

print(top_ceps.head(15))

           CEP                 Bairro  renda_mediana_2025  \
82   04753-060            Santo Amaro        22636.359900   
72   04727-002            Santo Amaro        18074.722050   
63   04719-050         Granja Julieta        26215.791525   
55   04717-004  Chácara Santo Antônio        16986.642750   
84   04755-070            Santo Amaro        16676.213400   
272  05708-290           Vila Andrade        25081.824075   
35   04708-001      Jardim Petrópolis        17478.776700   
8    04567-110          Brooklin Novo        32052.220200   
74   04730-990         Granja Julieta        18504.491775   
43   04711-130          Brooklin Novo        28064.819475   
42   04710-190  Chácara Santo Antônio        18047.706600   
295  05749-260           Vila Andrade        13511.270850   
287  05727-160           Paraisópolis        15878.327850   
85   04757-000            Santo Amaro        14798.737800   
286  05726-130           Paraisópolis        19375.633200   

     mediana_criancas_0

In [11]:
# Arredondar para 2 casas decimais (padrão monetário)
top_ceps["renda_mediana_2025"] = top_ceps["renda_mediana_2025"].round(2)
top_ceps["score_trafego_2025"] = top_ceps["score_trafego_2025"].round(2)

In [12]:
# Salvar resultado agregado na pasta do notebook (comportamento original)
top_ceps.to_csv("ElviraBrandao_Morumbi_top_ceps_2025.csv", index=False)

In [13]:
# Lista dos bairros desejados
bairros_desejados = ["Vila Sônia", "Ferreira", "Paraisópolis", "Jardim Maria Duarte", "Vila Andrade"]

# Filtrar diretamente no top_ceps
top_ceps_filtrado = top_ceps[top_ceps["Bairro"].isin(bairros_desejados)]

top_ceps_filtrado

,CEP,Bairro,renda_mediana_2025,mediana_criancas_0_9,populacao_mediana,score_trafego_2025
272,05708-290,Vila Andrade,25081.82,54.0,462.5,1354418.50
295,05749-260,Vila Andrade,13511.27,69.5,582.5,939033.32
287,05727-160,Paraisópolis,15878.33,58.0,383.0,920943.02
286,05726-130,Paraisópolis,19375.63,47.0,432.0,910654.76
271,05707-400,Vila Andrade,33537.00,23.0,267.0,771350.90
...,...,...,...,...,...,...
228,05625-160,Vila Sônia,8387.46,11.0,257.0,92262.06
123,05525-050,Ferreira,4793.27,19.0,288.0,91072.19
114,05520-100,Vila Sônia,3783.21,19.0,238.0,71881.07
118,05521-000,Vila Sônia,5049.91,11.0,248.0,55549.06


In [14]:
top_ceps_filtrado.to_csv("ElviraBrandao_Morumbi_top_ceps_filtrados_2025.csv", index=False)